# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gvts
# Import API key
from api_keys import geoapify_key

C:\Users\pinkd\anaconda3\envs\pythondata\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,25.99,78,8,4.12,CC,1723251181
1,1,rio grande,-32.0350,-52.0986,10.69,78,100,8.42,BR,1723251182
2,2,northam,51.0333,-4.2167,16.45,92,100,5.18,GB,1723251183
3,3,warman,52.3168,-106.5679,21.95,37,75,3.09,CA,1723251184
4,4,rongelap,11.1635,166.8941,28.49,75,100,7.62,MH,1723251186


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,
    size='humidity',
    color='City',
    tiles=gvts.CartoLight,
    frame_width=800, 
    frame_height=600
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp=city_data_df.loc[city_data_df['Max Temp'] <= 7].dropna()

# Drop any rows with null values
#dropna

# Display sample data
ideal_temp

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,grytviken,-54.2811,-36.5092,-10.24,77,100,1.97,GS,1723251199
23,23,port-aux-francais,-49.3500,70.2167,1.92,77,8,16.50,TF,1723251206
30,30,ushuaia,-54.8000,-68.3000,0.81,86,0,2.06,AR,1723251154
41,41,tiksi,71.6872,128.8694,4.86,93,100,3.97,RU,1723251226
58,58,olonkinbyen,70.9221,-8.7187,4.77,97,100,7.08,SJ,1723251246
68,68,puerto natales,-51.7236,-72.4875,1.81,99,100,0.27,CL,1723251256
94,94,lobujya,27.9500,86.8167,3.50,92,100,0.44,NP,1723251285
107,107,qaqortoq,60.7167,-46.0333,5.03,95,98,1.00,GL,1723251298
110,110,aasiaat,68.7098,-52.8699,2.60,96,100,1.82,GL,1723251302
113,113,aykhal,66.0000,111.5000,6.49,87,100,6.52,RU,1723251305


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_temp.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
17,grytviken,GS,-54.2811,-36.5092,77,
23,port-aux-francais,TF,-49.3500,70.2167,77,
30,ushuaia,AR,-54.8000,-68.3000,86,
41,tiksi,RU,71.6872,128.8694,93,
58,olonkinbyen,SJ,70.9221,-8.7187,97,
68,puerto natales,CL,-51.7236,-72.4875,99,
94,lobujya,NP,27.9500,86.8167,92,
107,qaqortoq,GL,60.7167,-46.0333,95,
110,aasiaat,GL,68.7098,-52.8699,96,
113,aykhal,RU,66.0000,111.5000,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10,000
params = {
    'Lat': -35.4323,
    'Lng':-60.1727,
    'categories': 'hotel',
    'apiKey': 'geoapify_key',
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    params["Lat"] = row['Lat']
    params["Lng"] = row['Lng']

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    


    # Convert the API response to JSON format
    name_address = name_address.json()	

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
lobujya - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
aasiaat - nearest hotel: No hotel found
aykhal - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
nuuk - nearest hotel: No hotel found
revda - nearest hotel: No hotel found
nagqu - nearest hotel: No hotel found
rufino - nearest hotel: No hotel found
veinticinco de mayo - nearest hotel: No hotel found
castillos - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
17,grytviken,GS,-54.2811,-36.5092,77,No hotel found
23,port-aux-francais,TF,-49.3500,70.2167,77,No hotel found
30,ushuaia,AR,-54.8000,-68.3000,86,No hotel found
41,tiksi,RU,71.6872,128.8694,93,No hotel found
58,olonkinbyen,SJ,70.9221,-8.7187,97,No hotel found
68,puerto natales,CL,-51.7236,-72.4875,99,No hotel found
94,lobujya,NP,27.9500,86.8167,92,No hotel found
107,qaqortoq,GL,60.7167,-46.0333,95,No hotel found
110,aasiaat,GL,68.7098,-52.8699,96,No hotel found
113,aykhal,RU,66.0000,111.5000,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

**The results always say 'No Hotel Found' even when I change the coordinates and preferences**